In [ ]:
import cadquery as cq 
from jupyter_cadquery import set_defaults, set_sidecar, get_defaults
from jupyter_cadquery.cadquery import show, PartGroup, Part, Faces, Edges
from jupyter_cadquery.cad_animation import Animation
from cadquery_massembly import Mate, MAssembly, relocate

# remove "clean" to avoid errors OCP kernel error
cq.occ_impl.shapes.Shape.clean = lambda x: x

set_sidecar("DiskArm", init=True)
set_defaults(axes=True, axes0=True, edge_accuracy=0.01, mate_scale=4, zoom=3.5, bb_factor=1.2)

In [ ]:
from math import pi, sin, cos, sqrt, asin, degrees, radians
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%matplotlib inline

# Disk and Arm

## Model

![disk-arm](1-disk-arm.png)

In [ ]:
r_disk = 100
dist_pivot = 200

def angle_arm(angle_disk):
    ra = np.deg2rad(angle_disk)
    v = np.array((dist_pivot, 0)) - r_disk * np.array((cos(ra), sin(ra)))
    return np.rad2deg(np.arctan2(*v[::-1]))

## Visualisation

In [ ]:
def diagram(da, aa, i1, i2):
    a1, a2 = radians(da), radians(aa)
    r1, r2 = r_disk, dist_pivot + r_disk
    x1, y1 = (0, r1 * cos(a1)), (0, r1 * sin(a1))
    x2, y2 = (dist_pivot, dist_pivot - r2 * cos(a2)), (0, -r2 * sin(a2))

    ax = fig.add_subplot(spec[i1, i2])
    ax.set_ylim(-r_disk-10, r_disk+10)
    ax.set_title(f"a={da}")

    ax.plot((0, dist_pivot), (0,0), color="lightgrey")
    ax.plot(x1, y1)
    ax.plot(x2, y2, ":")
    
    circle = plt.Circle((0,0), 100, fill=False, color="lightgrey", linestyle='--')
    ax.add_patch(circle)

In [ ]:
disk_angles = range(0, 360, 45)
arm_angles = [angle_arm(d) for d in disk_angles]

fig = plt.figure(constrained_layout=True)
fig.set_size_inches(20, 1.75)
spec = gridspec.GridSpec(ncols=8, nrows=1, figure=fig)

for i, (da, aa) in enumerate(zip(disk_angles, arm_angles)):
    diagram(da, aa, 0, i)

# Assembly

## Parts

In [ ]:
thickness = 5
nr = 5

disk = cq.Workplane().circle(r_disk + 2 * nr).extrude(thickness)
nipple = cq.Workplane().circle(nr).extrude(thickness)
disk = (
    disk
    .cut(nipple)
    .union(nipple.translate((r_disk, 0, thickness)))
)

pivot_base = cq.Workplane().circle(2*nr).extrude(thickness)
base = (
    cq.Workplane()
    .rect(6 * nr + dist_pivot, 6 * nr)
    .extrude(thickness)
    .translate((dist_pivot / 2, 0, 0))
    .union(nipple.translate((dist_pivot, 0, thickness)))
    .union(pivot_base.translate((0, 0, thickness)))
    .union(nipple.translate((0, 0, 2*thickness)))
    .edges("|Z").fillet(3)
)
base.faces(">Z[-2]").wires(cq.NearestToPointSelector((dist_pivot + r_disk, 0))).tag("mate")

slot = (cq.Workplane()
    .rect(2*r_disk, 2*nr)
    .extrude(thickness)
    .union(nipple.translate((-r_disk, 0, 0)))
    .union(nipple.translate((r_disk, 0, 0)))
    .translate((dist_pivot, 0, 0))
)
                                                                        
arm = (
    cq.Workplane()
    .rect(4 * nr + (r_disk + dist_pivot), 4 * nr)
    .extrude(thickness)
    .edges("|Z").fillet(3)    
    .translate(((r_disk + dist_pivot) / 2, 0, 0))
    .cut(nipple)
    .cut(slot)
)
arm.faces(">Z").wires(cq.NearestToPointSelector((0,0))).tag("mate")

show(
    disk,
    base.translate((0, -1.5 * r_disk, 0)),
    arm.translate((0, 1.5 * r_disk, 0)),
)

## Define assembly

In [ ]:
def create_disk_arm():
    L = lambda *args: cq.Location(cq.Vector(*args))
    C = lambda *args: cq.Color(*args)

    return (MAssembly(base, name="base", color=C("gray"), loc=L(-dist_pivot/2, 0, 0))
        .add(disk, name="disk", color=C("MediumAquaMarine"), loc=L(r_disk, -1.5 * r_disk, 0))
        .add(arm,  name="arm",  color=C("orange"), loc=L(0,  10*nr, 0))
    )

## Define mates

In [ ]:
from collections import OrderedDict as odict

disk_arm = create_disk_arm()

disk_arm.mate("base?mate", name="disk_pivot", origin=True, transforms=odict(rz=180))
disk_arm.mate("base@faces@>Z", name="arm_pivot")
disk_arm.mate("disk@faces@>Z[-2]", name="disk", origin=True)
disk_arm.mate("arm?mate", name="arm", origin=True)

show(disk_arm, render_mates=True)

## Relocate and assemble

In [ ]:
# ensure all parts are relocated so that the origin mates is the part origin 
relocate(disk_arm)

# assemble each part
disk_arm.assemble("arm", "arm_pivot")
disk_arm.assemble("disk", "disk_pivot")

d = show(disk_arm, render_mates=True, axes=False)

# Animate

In [ ]:
animation = Animation(d.root_group)

times = np.linspace(0, 5, 181)
disk_angles = np.linspace(0, 360, 181)
arm_angles = [angle_arm(d) for d in disk_angles]

# move disk
# Note, the selector must follow the path in the CAD view navigation hierarchy
animation.add_track(f"base/disk", "rz", times, disk_angles)

# move arm
animation.add_track(f"base/arm", "rz", times, arm_angles)

animation.animate(speed=2)